In [1]:
import gc, json, torch, logging, os
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from llmcompressor.modifiers.smoothquant import SmoothQuantModifier
from llmcompressor.modifiers.quantization import GPTQModifier
from llmcompressor import oneshot
from llmcompressor import configure_logger, LoggerConfig

from evaluation_utils import evaluate_mmlu, mmlu_harness_hf, display_metric

configure_logger(LoggerConfig(
    disabled=True,
    clear_loggers=True,
    console_log_level=None,
    log_file=None,
    log_file_level=None
))


In [2]:
hub_prefix = "TheS3b/Qwen3-0.6B-SmoothQuant-W8A8-calib"  # base name for HF pushes
model_repo = "brygotti/MNLP_M3_mcqa_model"
BITS         = 8
BLOCK_SIZE   = 64
prompt_sizes = [20, 200, 2000]
MAX_SEQUENCE_LENGTH = 2048

logging.disable(logging.INFO)
os.environ["EXLLAMA_KERNELS_AVAILABLE"] = "0"

all_metrics = {}

In [3]:
# Tokenizer once
tokenizer = AutoTokenizer.from_pretrained(model_repo, trust_remote_code=True)

filtered_calibration_set = load_dataset("TheS3b/MNLP_M3_quantized_dataset")["train"]

def tokenise(sample):
    return tokenizer(
        sample["prompt"],
        padding=False,
        max_length=MAX_SEQUENCE_LENGTH,
        truncation=True,
        add_special_tokens=False,
    )

eval_ds = load_dataset("brygotti/NLP4Education_english_single_mcq_4_choices")["test"]

for size in prompt_sizes:
    print(f"\n── SmoothQuant W{BITS}A8 with {size} calibration prompts ──\n")
    gc.collect()
    torch.cuda.empty_cache()

    # Build tokenised calibration dataset
    calib_ds = filtered_calibration_set.select(range(size)).map(
        tokenise, remove_columns=filtered_calibration_set.column_names
    )

    # FP16 baseline model
    model = AutoModelForCausalLM.from_pretrained(
        model_repo,
        torch_dtype=torch.float16,
        device_map="cuda",
        trust_remote_code=True,
    )

    recipe = [
        SmoothQuantModifier(
            smoothing_strength=0.8,
            ignore=["lm_head"],
            num_calibration_steps=size,
            block_size=BLOCK_SIZE,
        ),
        GPTQModifier(
            scheme=f"W{BITS}A8",
            targets="Linear",
            ignore=["lm_head"],
            block_size=BLOCK_SIZE,
        ),
    ]

    # One-shot quantisation pass
    oneshot(
        model=model,
        dataset=calib_ds,
        recipe=recipe,
        max_seq_length=MAX_SEQUENCE_LENGTH,
        num_calibration_samples=size,
    )

    model.eval()
    device = next(model.parameters()).device
    metrics = evaluate_mmlu(eval_ds, model, tokenizer, device, mmlu_harness_hf)
    display_metric(f"SmoothQuant W{BITS}A8 Size {size}", metrics)
    key = f"SmoothQuant W{BITS}A8 calib{size}"
    all_metrics[key] = metrics

    push_name = f"{hub_prefix}{size}"
    tokenizer.push_to_hub(push_name)
    model.push_to_hub(push_name)

    with open("Results/smooth_quant_metrics_calibration.json", "w") as f:
        json.dump(all_metrics, f, indent=2)

    del model
    gc.collect()
    torch.cuda.empty_cache()



── SmoothQuant W8A8 with 20 calibration prompts ──



20it [00:01, 16.37it/s]
Evaluating: 100%|██████████| 1962/1962 [22:53<00:00,  1.43it/s]



**SmoothQuant W8A8 Size 20 Evaluation Results**
- Accuracy              :  46.13 %
- Avg. inference time   :  699.2 ms
- Throughput (tok/s)    :  596.7
- Avg. peak VRAM        : 1207.4 MB
- Score Acc/VRAM        :  0.382 



Quantized Compression: 100%|██████████| 703/703 [00:02<00:00, 239.51it/s]


model.safetensors:   0%|          | 0.00/1.06G [00:00<?, ?B/s]

...

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

2000it [01:17, 25.78it/s]
Evaluating: 100%|██████████| 1962/1962 [23:25<00:00,  1.40it/s]



**SmoothQuant W8A8 Size 2000 Evaluation Results**
- Accuracy              :  46.48 %
- Avg. inference time   :  715.1 ms
- Throughput (tok/s)    :  583.5
- Avg. peak VRAM        : 2642.0 MB
- Score Acc/VRAM        :  0.176 



Quantized Compression: 100%|██████████| 703/703 [00:02<00:00, 267.27it/s]


model.safetensors:   0%|          | 0.00/1.06G [00:00<?, ?B/s]

In [4]:
hub_prefix = "TheS3b/Qwen3-0.6B-SmoothQuant-W4A8-calib"  # base name for HF pushes
model_repo = "brygotti/MNLP_M3_mcqa_model"
BITS         = [4]
BLOCK_SIZE   = 64
size = 200
MAX_SEQUENCE_LENGTH = 2048

logging.disable(logging.INFO)
os.environ["EXLLAMA_KERNELS_AVAILABLE"] = "0"

all_metrics = {}

In [5]:
# Tokenizer once
tokenizer = AutoTokenizer.from_pretrained(model_repo, trust_remote_code=True)

# Re-use the calibration set you already filtered
filtered_calibration_set = load_dataset("TheS3b/MNLP_M3_quantized_dataset")["train"]

# Tokenisation helper for llm-compressor (expects tokenised samples)
def tokenise(sample):
    return tokenizer(
        sample["prompt"],
        padding=False,
        max_length=MAX_SEQUENCE_LENGTH,
        truncation=True,
        add_special_tokens=False,
    )

eval_ds = load_dataset("brygotti/NLP4Education_english_single_mcq_4_choices")["test"]
calib_ds = filtered_calibration_set.select(range(size)).map(
    tokenise, remove_columns=filtered_calibration_set.column_names
)

for BIT in BITS:
    print(f"\n-- SmoothQuant W{BIT}A8 with {size} calibration prompts --\n")
    gc.collect()
    torch.cuda.empty_cache()

    # FP16 baseline model (no quantisation yet)
    model = AutoModelForCausalLM.from_pretrained(
        model_repo,
        torch_dtype=torch.float16,
        device_map="cuda",
        trust_remote_code=True,
    )

    recipe = [
        SmoothQuantModifier(
            smoothing_strength=0.8,
            ignore=["lm_head"],
            num_calibration_steps=size,
            block_size=BLOCK_SIZE,
        ),
        GPTQModifier(
            scheme=f"W{BIT}A8",
            targets="Linear",
            ignore=["lm_head"],
            block_size=BLOCK_SIZE,
        ),
    ]

    # One-shot quantisation pass
    oneshot(
        model=model,
        dataset=calib_ds,
        recipe=recipe,
        max_seq_length=MAX_SEQUENCE_LENGTH,
        num_calibration_samples=size,
    )

    model.eval()
    device = next(model.parameters()).device
    metrics = evaluate_mmlu(eval_ds, model, tokenizer, device, mmlu_harness_hf)
    display_metric(f"SmoothQuant W{BIT}A8 Size {size}", metrics)
    key = f"SmoothQuant W{BIT}A8 calib{size}"
    all_metrics[key] = metrics

    push_name = f"{hub_prefix}{size}"
    tokenizer.push_to_hub(push_name)
    model.push_to_hub(push_name)

    with open("Results/smooth_quant_metrics_bits.json", "w") as f:
        json.dump(all_metrics, f, indent=2)

    del model
    gc.collect()
    torch.cuda.empty_cache()



-- SmoothQuant W4A8 with 200 calibration prompts --



200it [00:07, 25.53it/s]
Evaluating: 100%|██████████| 1962/1962 [1:23:28<00:00,  2.55s/it]



**SmoothQuant W4A8 Size 200 Evaluation Results**
- Accuracy              :  43.12 %
- Avg. inference time   : 2551.4 ms
- Throughput (tok/s)    :  163.5
- Avg. peak VRAM        : 2651.0 MB
- Score Acc/VRAM        :  0.163 



Quantized Compression: 100%|██████████| 703/703 [00:16<00:00, 42.64it/s]


model.safetensors:   0%|          | 0.00/1.07G [00:00<?, ?B/s]